In [ ]:
import os
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import random
!pip install tensorflow_addons
import tensorflow_addons as tfa
import cv2
import numpy as np

# model

In [ ]:
src = '/content/drive/MyDrive/bdc 2021/Data/'
dest = '/content/drive/MyDrive/bdc 2021/cleaned_data/'

In [ ]:
train_dir = os.path.join(dest, 'train')
test_dir = os.path.join(dest, 'test')
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    # rotation_range = 10,
    # horizontal_flip = True,
    # shear_range = 0.2,
    # fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)


#train_datagen = ImageDataGenerator(
   # rescale = 1./255,
   # rotation_range = 40,
   # horizontal_flip = True,
   # shear_range = 0.2,
   # fill_mode = 'nearest',
   # zoom_range = 0.3,
   # width_shift_range = 0.2,
   # height_shift_range = 0.2
#)
train_datagen = cv2.fastNlMeansDenoisingColored(np.array(ImageDataGenerator), None, 3,3,7,21)

keras.preprocessing.image.ImageDataGenerator

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150, 150),
    batch_size = batch_size,
    class_mode = 'binary'
)

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size = (150, 150),
    batch_size = 4,
    class_mode = 'binary'
)

Found 2078 images belonging to 2 classes.
Found 232 images belonging to 2 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_generator.class_indices

{'0': 0, '1': 1}

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tfa.metrics.F1Score(1)])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached more than 96.0% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
# latih model dengan model.fit 
model.fit(
      train_generator,
      steps_per_epoch = train_generator.samples / batch_size,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs = 150, # tambahkan eposchs jika akurasi model belum optimal
      validation_data = test_generator, # menampilkan akurasi pengujian data validasi
      validation_steps = test_generator.samples / 4,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/150
129/129 - 72s - loss: 0.6252 - accuracy: 0.6492 - precision_4: 0.6044 - recall_4: 0.4719 - f1_score: 0.5907 - val_loss: 0.6520 - val_accuracy: 0.6034 - val_precision_4: 0.5200 - val_recall_4: 0.7959 - val_f1_score: 0.5939
Epoch 2/150
129/129 - 71s - loss: 0.6223 - accuracy: 0.6473 - precision_4: 0.5948 - recall_4: 0.4971 - f1_score: 0.5907 - val_loss: 0.6391 - val_accuracy: 0.6164 - val_precision_4: 0.6552 - val_recall_4: 0.1939 - val_f1_score: 0.5939
Epoch 3/150
129/129 - 72s - loss: 0.6228 - accuracy: 0.6526 - precision_4: 0.6054 - recall_4: 0.4914 - f1_score: 0.5907 - val_loss: 0.6212 - val_accuracy: 0.6638 - val_precision_4: 0.5962 - val_recall_4: 0.6327 - val_f1_score: 0.5939
Epoch 4/150
129/129 - 72s - loss: 0.6257 - accuracy: 0.6506 - precision_4: 0.5978 - recall_4: 0.5086 - f1_score: 0.5907 - val_loss: 0.6281 - val_accuracy: 0.6509 - val_precision_4: 0.5955 - val_recall_4: 0.5408 - val_f1_score: 0.5939
Epoch 5/150
129/129 - 72s - loss: 0.6232 - accuracy: 0.6444 - pr

In [ ]:
model.save('/content/drive/MyDrive/bdc 2021/model_v3.h5')

# Prediction

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image
import numpy as np

test_images = os.listdir(os.path.join(src, 'Testing'))
pred = []

for file in test_images:
  path = os.path.join(src, 'Testing', file)
  img = image.load_img(path, target_size=(150,150))
# imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = x/255.0
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model(images).numpy()

  if classes[0][0] > 0.5:
    pred.append(1)
  else:
    pred.append(0)

In [ ]:
df_temp = pd.DataFrame({'id': test_images, 'jenis kelamin': pred})
df_submission = pd.read_csv('/content/drive/MyDrive/bdc 2021/Data/submission.csv')

df_temp['id'] = df_temp['id'].apply(lambda x: x[:-4])

df_submission = df_submission.merge(df_temp, on='id', how='left')
df_submission

In [ ]:
df_submission.isnull().sum()

In [ ]:
df_submission['jenis kelamin'].value_counts()

In [ ]:
pd.DataFrame.to_csv(df_submission, '/content/drive/MyDrive/bdc 2021/submission_3.csv',index=False)

# Evaluation

In [ ]:
model =  tf.keras.models.load_model("/content/drive/MyDrive/bdc 2021/model_v2.h5")

In [ ]:
model.evaluate(test_generator)

In [ ]:
actual_label = pd.read_csv("/content/drive/MyDrive/bdc 2021/submission_labelled - Sheet1.csv")
predict_label = pd.read_csv("/content/drive/MyDrive/bdc 2021/submission_3.csv")

In [ ]:
actual_label.isnull().sum()

In [ ]:
actual_label['label'].value_counts()

In [ ]:
predict_label['jenis kelamin'].value_counts()

In [ ]:
actual_label.fillna(1, inplace=True)

In [ ]:
# checking whether or not id is equal
for i in range(len(predict_label)):
  if predict_label['id'][i] != actual_label['id'][i]:
    print(i)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(actual_label['label'].values, predict_label['jenis kelamin'].values))

In [ ]:
print(classification_report(actual_label['label'].values, df_submission['jenis kelamin'].values))